# 5. Optimization and Root Finding 
Many estimation problems in econometrics and statistics are essentially optimization problems, which in turn are reduced to root finding (e.g., F.O.C. for smooth objective/criterion functions). The Scipy module contains a number of routines to find the extremum/roots of a user-supplied objective function located in "scipy.optimize".

In [1]:
import scipy.optimize as opt                                     
import numpy as np

## 5.0 Root Finding
### 5.0.1 Univariate Roots
To find univariate roots of $f(x)$, "brentq( )" is the recommended method, which is a combination of bisection, secant (safe version) and inverse quadratic interpolation. "brentq( )" is a "bracketed" method which requires two initial values $a$, $b$ such that $f(a)f(b)<0$.

In [2]:
def f(x):
    return x**2 - 2*x -3

opt.brentq(f, -2, 0), opt.brentq(f, 0, 4)


(-0.9999999999999986, 2.999999999999924)

Univariate roots of $f(x)$ can also be found using Newton-Rhapson method (if $f'$ is provided via parameter "fprime") and secent method (otherwise) by calling "newton( )".

In [3]:
opt.newton(f, -2), opt.newton(f, 4, fprime = lambda x: 2*x-3)

(-1.0000000000000084, 3.0)

### 5.0.2 Multivariate Roots
Consider the task to solve a system of $m$ equations with $k$ unknowns. In practice, "root( )" is preferred for polynomial equations and "fsolve( )" is used for non-polynomial equations. Note that nonlinear system may have multiple solutions(or fixed points). 

In [4]:
def f(z):
    x, y = z[0], z[1]
    return [x**2 - x*y + y - 1,
            y**2 - x*y + x - 1]

sol1, sol2 = opt.root(f, (0,0)), opt.root(f, (-1,1))
print(sol1.x, sol2.x)
opt.fsolve(f, (0,0)), opt.fsolve(f, (-1,1))


[ 1.  1.] [ -2.64805212e-17   1.00000000e+00]


(array([ 1.,  1.]), array([ -2.64805212e-17,   1.00000000e+00]))

## 5.1 Unconstrained Optimization
A typical optimization problem in econometrics and statistics has an objective/criterion function that returns the function value at a set of parameters for given data (e.g., a log-likelihood function). Scipy optimizers:
1. require the objective function having the parameters as the first argument, and
2. search the minimum of the objective function.

### 5.1.1 Derivative-Based Methods
This class of optimizers require derivative and/or Hessian information, and so they can be applied only to problems with (sufficiently) smooth objective functions.
#### 5.1.1.1 BFGS (Broyden-Fletcher-Goldfarb-Shanno) Algorithm
BFGS is a "quasi" Newton method of optimization, which requires only first derivative ("first order method"). The Hessian $H$ is replaced by some numerical approximation. BFGS should usually be the first optimizer explored for unconstrained problems. Analytical first derivative can be provided using the keyword "fprime", otherwise a numerical approximation is used. In what follows, we can show the application of the BFGS optimizer "fmin_bfgs( )" (and other optimizers) using the Logit model below:

\begin{eqnarray*}
y_{i} & = & \mathbf{1}[\alpha_{0}+x_{i}'\beta_{0}+\epsilon_{i}>0]
\end{eqnarray*}

with $\epsilon_{i}\sim logistic(0,1)$ and true parameters $\alpha_{0}=0$,
$\beta_{0}=(1,1)$.

In [5]:
# simulate a Logit model
a, b = 0, np.array([[1.0],[1]])
np.random.seed(1) 
x, e = np.random.randn(5000,2), np.random.logistic(0,1,(5000,1))
u = a + x@b + e
y = 1*(u >= 0)
data = np.hstack((y, x))

# define the log-likelihood function 
def llh(params, hyperparams):
    theta = params
    data = hyperparams
    a, b = theta[0], np.vstack((theta[1], theta[2]))
    y, x = np.array(data[:,0:1]), np.array(data[:,1:data.shape[1]])
    v = a + x@b
    lh = y*np.log(np.exp(v)/(1+np.exp(v))) + (1-y)*np.log(1/(1+np.exp(v)))    
    return (-1)*sum(lh)

opt.fmin_bfgs(llh, np.random.randn(3,1),
              args = (data,),
              disp = True, maxiter = 1e6,
              retall = True, full_output = True)

# line 1: objective function, initial values
# line 2: tuple containing hyperparams (i.e., data), use a "," to make it a tuple
# line 3: display convergence message, maximum number of iterations
# line 4: return results of each iteration, return function value, gradient, Hessian, warnflag, etc


         Current function value: 2723.343344
         Iterations: 15
         Function evaluations: 471
         Gradient evaluations: 91


(array([ 0.01287582,  0.98580862,  0.99420631]),
 2723.3433441424527,
 array([ 0.00015259,  0.00021362,  0.00030518]),
 array([[  1.39439459e-04,   2.46009027e-05,   3.49182827e-05],
        [  2.46009027e-05,   8.48280400e-06,   8.47571983e-06],
        [  3.49182827e-05,   8.47571983e-06,   1.10655070e-05]]),
 471,
 91,
 2,
 [array([-0.79416259,  0.73143665, -0.17109954]),
  array([-0.16186972,  0.85393667,  0.6069104 ]),
  array([-0.20588847,  0.84016931,  0.64881368]),
  array([-0.21515835,  0.86919841,  0.65390003]),
  array([-0.18938382,  0.87919047,  0.68900246]),
  array([-0.13470045,  0.90147508,  0.76541822]),
  array([-0.00604669,  0.96116092,  0.95173956]),
  array([ 0.01158497,  0.98214998,  0.98929668]),
  array([ 0.01290723,  0.98566261,  0.99407507]),
  array([ 0.01288101,  0.98580677,  0.99420653]),
  array([ 0.0128765 ,  0.98580865,  0.99420584]),
  array([ 0.01287625,  0.98580859,  0.99420589]),
  array([ 0.01287606,  0.98580866,  0.99420602]),
  array([ 0.01287583, 

#### 5.1.1.2 Conjugate Gradient Methods
As an alternative to "fmin_bfgs( )", "fmin_cg( )" uses a nonlinear conjugate gradient method to find the minimum. Analytical first derivative can be provided, otherwise it is numerically approximated. "fmin_ncg( )" use a Newton conjugate gradient method and it requires a function which can compute the first derivative, though providing the Hessian is optional.

In [6]:
opt.fmin_cg(llh, np.random.randn(3,1), args = (data,), disp = True)

         Current function value: 2723.343344
         Iterations: 11
         Function evaluations: 293
         Gradient evaluations: 57


array([ 0.01287565,  0.9858081 ,  0.99420605])

### 5.1.2 Derivative-Free Optimization
Derivative-free optimizers do not use derivative information and so can be used in a wider variety of problems such as
functions which are not continuously differentiable. However, when applied to smooth objective functions, they are likely to be slower than derivative-based optimizers. An example for problems with an unsmooth objective function is the maximum score estimator for the Logit model which maximizes

\begin{eqnarray*}
Q(\alpha,\beta) & = & \sum_{i=1}^{n}\{y_{i}\mathbf{1}[\alpha+x_{i}'\beta>0]+(1-y_{i})\mathbf{1}[\alpha+x_{i}'\beta<0]\}
\end{eqnarray*}

#### 5.1.2.1 Nelder-Mead Simplex
While Newton's method is "second order" (i.e., requires the second derivative) and quasi-Newton methods are first order (requires only first derivatives), Nelder-Mead is a zero order method, i.e., it does not requires any derivatives and only uses the objective function itself. The Nelder-Mead simplex algorithm can be implemented using "fmin( )".

In [7]:
# objective function of maximum score estimation
def maxscore(params, hyperparams):
    theta = params
    data = hyperparams
    a, b = theta[0], np.vstack((theta[1], 1))
    y, x = np.array(data[:,0:1]), np.array(data[:,1:data.shape[1]])
    v = a + x@b
    score = y*(v >= 0) + (1-y)*(v < 0)    
    return (-1)*np.mean(score)

np.random.seed(1)
opt.fmin(maxscore, np.random.randn(2,1), args = (data,))

Optimization terminated successfully.
         Current function value: -0.726000
         Iterations: 42
         Function evaluations: 93


array([ 0.05367786,  0.92626561])

#### 5.1.2.2 Powell's Methods
Powell's method is a derivative-free optimization method similar to conjugate-gradient, which can be implemented using "fmin_powell( )". In practice, "fmin_powell( )" often converges faster (requires far fewer iterations) and is more reliable.

In [8]:
np.random.seed(1)
opt.fmin_powell(maxscore, np.random.randn(2,1), args = (data,))

Optimization terminated successfully.
         Current function value: -0.725200
         Iterations: 3
         Function evaluations: 142


array([ 0.09593936,  0.97424993])

## 5.2 Constrained Optimization
In econometrics and statistics, constrained optimization is often used in hypothesis testing problems. The most common analytical technique is the method of Lagrange multipliers. "fmin_slsqp( )" is the most general constrained optimizer using sequential least squares programming, which allows for equality, inequality and bounds constraints. To use it, constraints must be provided either as a list of callable functions or as a single function which returns an array. Functions which compute the derivative of the optimization target, the derivative of the equality constraints, and the derivative of the inequality constraints can be optionally provided. If not provided, these are numerically approximated. Note that for this optimizer,
1. Arguments in constraints must be identical to those of the objective function.
2. All constraints must take the form of $\geq$ constraint.

In [9]:
# simulate a NLS model
b = np.array([[1],[0.5]])
x, e = np.random.randn(2000,1), np.random.randn(2000,1)
y = np.exp(b[0] + b[1]*x) + e
data = np.hstack((y, x))

def nls(params, hyperparams):
    b = params
    data = hyperparams 
    y, x = np.array(data[:,0:1]), np.array(data[:,1:2])
    dy = y - np.exp(b[0] + b[1]*x)       
    return np.mean(dy**2)
   
def nls_constraints(params, hyperparams): 
    b = params
    return np.array([b[1], b[0]-2*b[1], 2*b[1]-b[0]])

# optimization with constraints
opt.fmin_slsqp(nls, np.random.randn(2,1), args = (data,),
               f_ieqcons = nls_constraints)                   


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.9806773073767117
            Iterations: 7
            Function evaluations: 30
            Gradient evaluations: 7


array([ 0.99717628,  0.49858814])

In [10]:
opt.fmin_bfgs(nls, np.random.randn(2,1), args = (data,))

Optimization terminated successfully.
         Current function value: 0.980668
         Iterations: 12
         Function evaluations: 72
         Gradient evaluations: 18


array([ 0.99827142,  0.49782124])

Note: All optimizers introduced here find only a $local$ minimum. To search for the $global$ minimum, users may try multiple (random) initial values.

### References for Optimization Algorithms
1. "Numerical Optimization", by J. Nodecal and S. Wright, Springer, 2006.
2. "Convex Optimization", by S. Boyd and L. Vandenberghe, Cambridge University Press, 2004.